# VV Correlation Heatmaps

Ravi Mandla

Code used to create correlation heatmaps of VV RNA-seq data

Code was taken from http://www.sthda.com/english/wiki/ggplot2-quick-correlation-matrix-heatmap-r-software-and-data-visualization

In [1]:
library(reshape2)
library(ggplot2)
library(readxl)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Functions

In [48]:
# Get lower triangle of the correlation matrix
  get_lower_tri<-function(cormat){
    cormat[upper.tri(cormat)] <- NA
    return(cormat)
  }
  # Get upper triangle of the correlation matrix
  get_upper_tri <- function(cormat){
    cormat[lower.tri(cormat)]<- NA
    return(cormat)
  }

reorder_cormat <- function(cormat){
# Use correlation between variables as distance
dd <- as.dist((1-cormat)/2)
hc <- hclust(dd)
cormat <-cormat[hc$order, hc$order]
}

Import data

In [62]:
data <- read_tsv("~/Downloads/normalized_counts.tsv")

Parsed with column specification:
cols(
  ID = col_character(),
  Name = col_character(),
  VV.Red._ = col_double(),
  VV.Red.P2 = col_double(),
  VV.Red.P4 = col_double(),
  VV.RedGreen._ = col_double(),
  VV.RedGreen.P2 = col_double(),
  VV.RedGreen.P4 = col_double()
)



In [63]:
norm_genename = data %>% select(3:8)

In [64]:
rownames(norm_genename) = data$...1

Warning message:
“Unknown or uninitialised column: `...1`.”


Make heatmap

In [65]:
cormat <- round(cor(norm_genename),2)

In [66]:
upper_tri <- get_upper_tri(cormat)

In [67]:
melted_cormat <- melt(upper_tri, na.rm = TRUE)

In [68]:
# Reorder the correlation matrix
cormat <- reorder_cormat(cormat)
upper_tri <- get_upper_tri(cormat)
# Melt the correlation matrix
melted_cormat <- melt(upper_tri, na.rm = TRUE)
# Create a ggheatmap
ggheatmap <- ggplot(melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
    name="Pearson\nCorrelation") +
  theme_minimal()+ # minimal theme
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()

In [69]:
png(file='corr-heatmap-rm.png')
ggheatmap + 
geom_text(aes(Var2, Var1, label = value), color = "black", size = 4) +
theme(
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.grid.major = element_blank(),
  panel.border = element_blank(),
  panel.background = element_blank(),
  axis.ticks = element_blank(),
  legend.justification = c(1, 0),
  legend.position = c(0.6, 0.7),
  legend.direction = "horizontal")+
  guides(fill = guide_colorbar(barwidth = 7, barheight = 1,
                title.position = "top", title.hjust = 0.5))
dev.off()

png 
  2

This heatmap is not interesting, so I will make one with only DE genes also.

In [8]:
fdr = read.table("~/Downloads/deseq2_res.csv",sep=',',header=TRUE)

In [15]:
sig = na.omit(fdr[fdr$padj <= .01, ])

In [38]:
sigensembl = sig$"Ensembl"

In [52]:
normvals = read_tsv("~/Downloads/normalized_counts.tsv")

Parsed with column specification:
cols(
  ID = col_character(),
  Name = col_character(),
  VV.Red._ = col_double(),
  VV.Red.P2 = col_double(),
  VV.Red.P4 = col_double(),
  VV.RedGreen._ = col_double(),
  VV.RedGreen.P2 = col_double(),
  VV.RedGreen.P4 = col_double()
)



In [57]:
vals = normvals %>% select(1,3:8)

In [58]:
sig_normvals = filter(vals, ID %in% sigensembl)

In [59]:
sig_normvals = select(sig_normvals,2:7)

In [60]:
cormat <- round(cor(sig_normvals),2)
upper_tri <- get_upper_tri(cormat)
melted_cormat <- melt(upper_tri, na.rm = TRUE)

# Reorder the correlation matrix
cormat <- reorder_cormat(cormat)
upper_tri <- get_upper_tri(cormat)
# Melt the correlation matrix
melted_cormat <- melt(upper_tri, na.rm = TRUE)
# Create a ggheatmap
ggheatmap <- ggplot(melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
    name="Pearson\nCorrelation") +
  theme_minimal()+ # minimal theme
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()

png(file='corr-heatmap-rm-de.png')
ggheatmap + 
geom_text(aes(Var2, Var1, label = value), color = "black", size = 4) +
theme(
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.grid.major = element_blank(),
  panel.border = element_blank(),
  panel.background = element_blank(),
  axis.ticks = element_blank(),
  legend.justification = c(1, 0),
  legend.position = c(0.6, 0.7),
  legend.direction = "horizontal")+
  guides(fill = guide_colorbar(barwidth = 7, barheight = 1,
                title.position = "top", title.hjust = 0.5))
dev.off()

png 
  2